In [1]:

from openai import OpenAI
import mysql.connector
from pydantic import BaseModel, Field
import json

client = OpenAI(api_key='')

In [2]:
indic_base= client.files.create(
    file= open('indic_insitel.pdf', 'rb'),
    purpose= 'user_data'
)
indic_lic1= client.files.create(
    file= open('indic1.pdf', 'rb'),
    purpose= 'user_data'
)
indic_lic2= client.files.create(
    file= open('indic2.pdf', 'rb'),
    purpose= 'user_data'
)
indic_lic3= client.files.create(
    file= open('indic3.pdf', 'rb'),
    purpose= 'user_data'
)

In [1]:
# Conexión a la base de datos:
class ConexionMySQL:
    def __init__(self, host, port, user, password, database):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.database = database

        # Aquí iría la lógica para conectarse a MySQL
        self.conexion = mysql.connector.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            database=self.database
        )
        self.cursor = self.conexion.cursor()
        if self.conexion.is_connected():
            # Verificar la conexión
            print("Conexión exitosa a la base de datos MySQL")
        else:
            print("Error al conectar a la base de datos MySQL")

In [ ]:
def main_mysql():
    # Crear una instancia de la clase ConexionMySQL
    db_mysql = ConexionMySQL(
        host="localhost",
        port=3306,
        user = "root",
        password = "admin",
        database = "db_insitel"
    )

    # Ejecutar una consulta a la base de datos
    db_mysql.cursor.execute("SELECT * FROM indicadores_financieros;")

    # Obtener los resultados
    resultados = db_mysql.cursor.fetchall()
    # Imprimir los resultados
    for fila in resultados:
        print(fila)

    # Cerrar la conexión
    db_mysql.cursor.close()
    db_mysql.conexion.close()
    print("Conexión cerrada")
main_mysql()

In [3]:
# Modelo:
class Indicadores(BaseModel):
    descrip: str = Field(description='Nombre del Indicador Solicitado:')
    indice_insitel: float = Field(description='Valor Indice en Base de Datos INSITEL:')
    indice_solicitado: float = Field(description='Valor Indice Solicitado en Licitacion:')
    cumple: bool = Field(description='Cumple / No Cumple:')
    valor3: float = Field(description='Si no cumplimos entonces sacar resta de valor indice solicitado menos valor indice en base de datos:')


In [4]:
# Prompt propuesto:
response = client.responses.create(
    model='gpt-4o-mini',
    input=[
        {
            'role': 'user',
            'content': [
                {'type': 'input_file', 'file_id': indic_base.id},
                {'type': 'input_file', 'file_id': indic_lic3.id},
                {'type': 'input_text', 'text': (
                    'Analiza los indicadores financieros entregados en el archivo base insitel.'
                    'y compáralos con los indicadores solicitados en el archivo de licitación, tener en cuenta la conversión de porcentaje a decimal.'
                    'Para cada indicador, responde solo en formato JSON (sin texto adicional), con esta estructura exacta:\n\n'
                    '[\n'
                    '  {\n'
                    '    "Indicador": "Nombre del indicador solicitado",\n'
                    '    "Valor Indicador Base INSITEL": indice_insitel,\n'
                    '    "Valor Indicador Solicitado": indice_solicitado,\n'
                    '    "Cumplimos": true/false\n'
                    '    "Valor Faltante": Si Cumplimos==false => valor_obtenido-valor_db,\n'
                    '  },\n'
                    '  ...\n'
                    ']\n\n'
                    "No incluyas explicaciones ni texto adicional. Solo el arreglo JSON con los indicadores."
                )}
            ]
        }
    ]
)

In [5]:
# Se extrae y parsea la respuesta
salida = response.output_text.strip()

# Se valida que sea un JSON válido antes de parsear
try:
    lista_dicts = json.loads(salida)
    indicadores = [Indicadores(**item) for item in lista_dicts]
    for ind in indicadores:
        print(ind)
except Exception as e:
    print("Error al convertir la salida en objetos Indicadores:", e)
    print("Respuesta del modelo:\n", salida)


Error al convertir la salida en objetos Indicadores: Expecting value: line 1 column 1 (char 0)
Respuesta del modelo:
 ```json
[
  {
    "Indicador": "Índice de liquidez",
    "Valor Indicador Base INSITEL": 2.16,
    "Valor Indicador Solicitado": 6.60,
    "Cumplimos": false,
    "Valor Faltante": -4.44
  },
  {
    "Indicador": "Índice de endeudamiento",
    "Valor Indicador Base INSITEL": 0.47,
    "Valor Indicador Solicitado": 0.60,
    "Cumplimos": true
  },
  {
    "Indicador": "Razón de cobertura de intereses",
    "Valor Indicador Base INSITEL": 19.83,
    "Valor Indicador Solicitado": 1.70,
    "Cumplimos": true
  },
  {
    "Indicador": "Patrimonio",
    "Valor Indicador Base INSITEL": null,
    "Valor Indicador Solicitado": 1,
    "Cumplimos": false,
    "Valor Faltante": null
  },
  {
    "Indicador": "Capital de Trabajo",
    "Valor Indicador Base INSITEL": null,
    "Valor Indicador Solicitado": 1,
    "Cumplimos": false,
    "Valor Faltante": null
  },
  {
    "Indicador"